In [ ]:
!pip install --quiet flashtext==2.7
!pip install git+https://github.com/boudinfl/pke.git
!pip install --quiet transformers==4.8.1
!pip install --quiet sentencepiece==0.1.95
!pip install --quiet textwrap3==0.9.2
!pip install --quiet gradio==3.0.20
!pip install --quiet strsim==0.0.3
!pip install --quiet sense2vec==2.0.0
!pip install --quiet ipython-autotime
!pip install --upgrade huggingface_hub
%load_ext autotime
!pip install --quiet sentence-transformers==2.2.2
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-h6en2sqk
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-h6en2sqk
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
--2024-12-20 06:12:26--  https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
Resolving githu

In [ ]:
pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


time: 2.32 s (started: 2024-12-20 06:12:43 +00:00)


In [ ]:
import anvil.server
anvil.server.connect("server_EATVENTPOGH7AH25767NPNAD-YAJNNTPIKXVPEX4O")
import os
import pke
import torch
import nltk
import random
import string
import gradio as gr
import numpy as np
from textwrap3 import wrap
from sense2vec import Sense2Vec
from nltk.corpus import stopwords
from collections import OrderedDict
from nltk.corpus import wordnet as wn
from flashtext import KeywordProcessor
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5ForConditionalGeneration, T5Tokenizer
from similarity.normalized_levenshtein import NormalizedLevenshtein

nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "summary_questions_distractors_generator" as SERVER


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/__init__.py)

time: 14.2 s (started: 2024-12-20 06:12:45 +00:00)


In [ ]:
import pke
import string
from nltk.corpus import stopwords

@anvil.server.callable
def ans(input_text):
  class TextSummarizer:
      def __init__(self, model_name='t5-base'):
          """Initialize model and tokenizer."""
          self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(self.get_device())
          self.tokenizer = T5Tokenizer.from_pretrained(model_name)

      def get_device(self):
          """Return the device GPU/CPU to use."""
          return torch.device("cuda" if torch.cuda.is_available() else "cpu")

      def summarize_text(self, text, max_length=500, min_length=100):
          text = text.strip().replace("\n", " ")
          text = "summarize: " + text
          encoding = self.tokenizer.encode_plus(text, max_length=512, pad_to_max_length=False, truncation=True, return_tensors="pt").to(self.get_device())
          outs = self.model.generate(input_ids=encoding["input_ids"],
                                    attention_mask=encoding["attention_mask"],
                                    early_stopping=True,
                                    num_beams=5,
                                    num_return_sequences=1,
                                    no_repeat_ngram_size=3,
                                    min_length=min_length,
                                    max_length=max_length)

          summary = self.tokenizer.decode(outs[0], skip_special_tokens=True)
          return self.post_process_text(summary)

      @staticmethod
      def post_process_text(content):
          final = ""
          for sent in sent_tokenize(content):
              final += " " + sent.capitalize()
          return final.strip()
  text_summarizer = TextSummarizer()
  sample_text = input_text
  # Input from anvil
  summarized_text = text_summarizer.summarize_text(sample_text)
  print("Summarized Text:", summarized_text)


  class KeywordExtractor:
      def __init__(self):
          self.keyword_processor = KeywordProcessor()

      @staticmethod
      def extract_nouns_multipartite(content):
          """
          Extracts nouns and proper nouns from the given content using MultipartiteRank algorithm.
          """
          out = []
          try:
              extractor = pke.unsupervised.MultipartiteRank()
              extractor.load_document(input=content, language='en')
              pos = {'PROPN', 'NOUN'}
              stoplist = list(string.punctuation) + stopwords.words('english')
              extractor.candidate_selection(pos=pos)
              extractor.candidate_weighting(alpha=1.5, threshold=0.80, method='average')
              keyphrases = extractor.get_n_best(n=25)

              for val in keyphrases:
                  out.append(val[0])
          except Exception:
              pass

          return out

      def get_keywords(self, original_text, summary_text):
          """
          Identifies important keywords from the original text and checks their presence in the summary.

          Parameters:
          original_text (str): The original text to extract keywords from.
          summary_text (str): The summarized text to check against.

          Returns:
          list: A list of important keywords found in both original and summarized text.
          """
          keywords = self.extract_nouns_multipartite(original_text)
          print('Keywords in unsummarized : ', keywords)
          for keyword in keywords:
              self.keyword_processor.add_keyword(keyword)

          keywords_found = list(set(self.keyword_processor.extract_keywords(summary_text)))
          print('Keywords in summarized : ', keywords_found)

          important_keywords = [keyword for keyword in keywords if keyword in keywords_found]

          return important_keywords[:4]

  keyword_extractor = KeywordExtractor()
  keywords = keyword_extractor.get_keywords(sample_text, summarized_text)
  print("Extracted Keywords:", keywords)
  class QuestionGenerator:
      def __init__(self, model_name='ramsrigouthamg/t5_squad_v1'):
          """
          Initializes the QuestionGenerator with a pre-trained model and tokenizer.
          """
          self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(self.get_device())
          self.tokenizer = T5Tokenizer.from_pretrained(model_name)

      def get_device(self):
          return torch.device("cuda" if torch.cuda.is_available() else "cpu")

      def generate_question(self, context, answer):
          """
          Generates questions and answers based on the given context and model.

          Parameters:
          context (str): The context for the question generation.
          answer (str): The answer to generate a question about.

          Returns:
          str: The generated question.
          """
          text = f"context: {context} answer: {answer}"
          encoding = self.tokenizer.encode_plus(text, max_length=384, pad_to_max_length=False, truncation=True, return_tensors="pt").to(self.get_device())

          outs = self.model.generate(input_ids=encoding["input_ids"],
                                    attention_mask=encoding["attention_mask"],
                                    early_stopping=True,
                                    num_return_sequences=2,
                                    no_repeat_ngram_size=3,
                                    num_beams=7,
                                    max_length=75)

          question = self.tokenizer.decode(outs[0], skip_special_tokens=True)
          return question.replace("question:", "").strip()
  class QuestionGenerator:
      def __init__(self, model_name='ramsrigouthamg/t5_squad_v1'):
          """
          Initializes the QuestionGenerator with a pre-trained model and tokenizer.
          """
          self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(self.get_device())
          self.tokenizer = T5Tokenizer.from_pretrained(model_name)

      def get_device(self):
          return torch.device("cuda" if torch.cuda.is_available() else "cpu")

      def generate_question(self, context, answer):
          """
          Generates questions and answers based on the given context and model.

          Parameters:
          context (str): The context for the question generation.
          answer (str): The answer to generate a question about.

          Returns:
          str: The generated question.
          """
          text = f"context: {context} answer: {answer}"
          encoding = self.tokenizer.encode_plus(text, max_length=384, pad_to_max_length=False, truncation=True, return_tensors="pt").to(self.get_device())

          outs = self.model.generate(input_ids=encoding["input_ids"],
                                    attention_mask=encoding["attention_mask"],
                                    early_stopping=True,
                                    num_return_sequences=2,
                                    no_repeat_ngram_size=3,
                                    num_beams=7,
                                    max_length=75)

          question = self.tokenizer.decode(outs[0], skip_special_tokens=True)
          return question.replace("question:", "").strip()

  question_generator = QuestionGenerator()
  questions = [question_generator.generate_question(summarized_text, keyword) for keyword in keywords]
  for question in questions:
    print("Generated Question:", question)

  class DistractorGenerator:
      def __init__(self, sense2vec_model, sentence_transformer_model):
          """
          Initializes the DistractorGenerator with Sense2Vec and Sentence Transformer models.
          """
          self.s2v = sense2vec_model
          self.sentence_model = sentence_transformer_model
          self.normalized_levenshtein = NormalizedLevenshtein()

      def sense2vec_get_words(self, word, topn, question):
          try:
              sense = self.s2v.get_best_sense(word, senses=["NOUN", "PERSON", "PRODUCT", "LOC", "ORG", "EVENT", "NORP", "WORK OF ART", "FAC", "GPE", "NUM", "FACILITY"])
              most_similar = self.s2v.most_similar(sense, n=topn)
              return self.filter_same_sense_words(sense, most_similar)
          except Exception:
              return []

      @staticmethod
      def filter_same_sense_words(original, wordlist):
          base_sense = original.split('|')[1]
          return [eachword[0].split('|')[0].replace("_", " ").title().strip() for eachword in wordlist if eachword[0].split('|')[1] == base_sense]

      def get_distractors(self, word, sentence, top_n=40, lambda_param=0.2):
          distractors = self.sense2vec_get_words(word, top_n, sentence)
          if not distractors:
              return []

          distractors = [word.capitalize()] + distractors

          sentence_embedding = self.sentence_model.encode([sentence + " " + word.capitalize()])
          distractor_embeddings = self.sentence_model.encode(distractors)

          filtered_keywords = self.mmr(sentence_embedding, distractor_embeddings, distractors, min(len(distractors), 5), lambda_param)

          return [wrd for wrd in filtered_keywords if wrd.lower() != word.lower()][1:]

      @staticmethod
      def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
          """
          Applies the MMR (Maximal Marginal Relevance) algorithm to select top distractor words.

          Parameters:
          doc_embedding (np.array): The embedding of the sentence context.
          word_embeddings (np.array): The embeddings of the distractor words.
          words (list): The list of distractor words.
          top_n (int): The number of top words to return.
          lambda_param (float): Parameter for balancing relevance and diversity.

          Returns:
          list: A list of top distractor words based on MMR.
          """
          word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
          word_similarity = cosine_similarity(word_embeddings)

          keywords_idx = [np.argmax(word_doc_similarity)]
          candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

          for _ in range(top_n - 1):
              candidate_similarities = word_doc_similarity[candidates_idx, :]
              target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

              mmr = (lambda_param * candidate_similarities) - ((1 - lambda_param) * target_similarities.reshape(-1, 1))
              mmr_idx = candidates_idx[np.argmax(mmr)]

              keywords_idx.append(mmr_idx)
              candidates_idx.remove(mmr_idx)

          return [words[idx] for idx in keywords_idx]

  s2v_model = Sense2Vec().from_disk('s2v_old')
  sentence_model = SentenceTransformer('msmarco-distilbert-base-v3')
  distractor_generator = DistractorGenerator(s2v_model, sentence_model)

  for keyword, question in zip(keywords, questions):
    distractors = distractor_generator.get_distractors(keyword, question)
    print(f"Distractors for {keyword}: {distractors}")

  class QuestionAnswerGeneratorWithGradio:
      """

      Attributes:
        text_summarizer:
        keyword_extractor:
        question_generator:
        s2v_model:
        sentence_model:
        distractor_generator:
      """
      def __init__(self):
          """
          Initializes the QuestionAnswerGeneratorWithGradio with necessary models for text summarization,
          keyword extraction, question generation, and distractor generation.
          """
          # Initialize the text summarizer
          self.text_summarizer = TextSummarizer()

          # Initialize the keyword extractor
          self.keyword_extractor = KeywordExtractor()

          # Initialize the question generator
          self.question_generator = QuestionGenerator()

          # Load the Sense2Vec model from disk
          self.s2v_model = Sense2Vec().from_disk('s2v_old')

          # Initialize the sentence transformer model
          self.sentence_model = SentenceTransformer('msmarco-distilbert-base-v3')

          # Initialize the distractor generator with the models
          self.distractor_generator = DistractorGenerator(self.s2v_model, self.sentence_model)

      def generate_questions_and_answers(self, content):
          summary = self.text_summarizer.summarize_text(content)
          keywords = self.keyword_extractor.get_keywords(content, summary)
          questions = [self.question_generator.generate_question(summary, keyword) for keyword in keywords]
          distractors = {keyword: self.distractor_generator.get_distractors(keyword, questions[i]) for i, keyword in enumerate(keywords)}
          return summary, questions, distractors

      def generate_output(self, content):
          summary, questions, distractors = self.generate_questions_and_answers(content)
          result = {
              "Summary": summary,
              "Questions and Distractors": [
                  {
                      "Question": questions[i],
                      "Distractors": distractors[keyword]
                  } for i, keyword in enumerate(keywords)
              ]
          }
          return result


  def qa_pipeline(input_text):
      """
      Processes the input text to generate a summary, questions, and distractors, and formats the output.

      Parameters:
      input_text (str): The input text content to process.

      Returns:
      str: A string containing the summary, questions, and their distractors.
      """

  s2v_model = Sense2Vec().from_disk('s2v_old')
  sentence_model = SentenceTransformer('msmarco-distilbert-base-v3')
  distractor_generator = DistractorGenerator(s2v_model, sentence_model)

  for keyword, question in zip(keywords, questions):
      distractors = distractor_generator.get_distractors(keyword, question)
      print(f"Distractors for {keyword}: {distractors}")


  qa_generator = QuestionAnswerGeneratorWithGradio()
  summary, questions, distractors = qa_generator.generate_questions_and_answers(sample_text)

  generator = QuestionAnswerGeneratorWithGradio()
  summary, questions, distractors = generator.generate_questions_and_answers(input_text)
  output = f"Summary:\n{summary}\n\n"
  for i, question in enumerate(questions):
    if i!=len(questions)-1:
      output += f"Q{i+1}: {question}\n"
      output += f"Distractors: {', '.join(distractors[keywords[i]])}\n\n"
  return output



In [ ]:
anvil.server.wait_forever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
